In [66]:
import numpy as np
import pandas as pd
import json

In [67]:
import psycopg2
def connect(database_name):
    return psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  database=database_name
                                  )

In [68]:
import logging
from psycopg2.extras import execute_values
def execute_bulk_insert(query,connection,data):
    
    try:
        cursor = connection.cursor()

        
        #data=[[str(ele) for ele in item.values()] for item in data]

#         data=[[ele for ele in item.values()] for item in data]
#         print(data[0])
        
        execute_values(
        cursor, query,data)

        connection.commit()
        print(query , "sucessfull")
    except (Exception,psycopg2.Error) as error:
        print(error)
        logging.debug("An exception was thrown!", exc_info=True)
        logging.info("An exception was thrown!", exc_info=True)
        logging.warning("An exception was thrown!", exc_info=True)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("connection closed")

In [69]:
def execute_insert_query(query,connection,data):
    """This is the method to execute the insert sql query given the parameter the query and connection method and data"""

    try:
        conn=connection
        cur=conn.cursor()
        for item in data:
            cur.execute(query,tuple(item))
        conn.commit()
        print(query+",Sucessfully inserted data")
    except(Exception) as e:
        print(e)
    finally:
        if(conn):
            cur.close()
            conn.close()

In [ ]:
import sys
import json


# from ..utils import connect
# from ..helper import execute_bulk_insert



def extract_data(file_path,query_path,database_name,batch_size):
    #print(database_name)
    with open(query_path,'r') as q:
        q=q.read()

    print(q)
    print(file_path)
    


    with open(file_path,'r') as f:
        data=[]
        i=0
        for line in f:
            data.append(json.loads(line))
            i+=1

            if i==batch_size:
                execute_bulk_insert(q,connect(database_name),data)
                i=0
                data=[]
                continue
        #execute the remaining data
        if data!=[]:
            execute_bulk_insert(q,connect(database_name),data)
        
        
            

if __name__=='__main__':
    database_name="yelp_db"
    batch_size=10000

    file_paths_and_query_path ={
                # 'user':['../../datasets/yelp_dataset/yelp_academic_dataset_user.json','../sql/insert_into_raw_user.sql'],
                'business':['../../datasets/yelp_dataset/yelp_academic_dataset_business.json','../sql/insert_into_raw_business.sql'],
                # 'photos':['../../datasets/yelp_dataset/photos.json','../sql/insert_into_raw_photo.sql'],
                # 'tip':['../../datasets/yelp_dataset/yelp_academic_dataset_tip.json','../sql/insert_into_raw_tip.sql'],
                # 'review':['../../datasets/yelp_dataset/yelp_academic_dataset_review.json','../sql/insert_into_raw_review.sql'],
                # 'checkin':['../../datasets/yelp_dataset/yelp_academic_dataset_checkin.json','../sql/insert_into_raw_checkin.sql']
                }
    
    for paths in file_paths_and_query_path.values():
        extract_data(paths[0],paths[1],database_name,batch_size)

In [75]:
path = "datasets/yelp_dataset/yelp_academic_dataset_business.json"
with open(path,'r') as f:
    data=[]
    i=0
    for line in f:
        data.append(json.loads(line))
        i+=1
        if i==1000:
            break


In [76]:
data[0]

{'business_id': '6iYb2HFDywm3zjuRg0shjw',
 'name': 'Oskar Blues Taproom',
 'address': '921 Pearl St',
 'city': 'Boulder',
 'state': 'CO',
 'postal_code': '80302',
 'latitude': 40.0175444,
 'longitude': -105.2833481,
 'stars': 4.0,
 'review_count': 86,
 'is_open': 1,
 'attributes': {'RestaurantsTableService': 'True',
  'WiFi': "u'free'",
  'BikeParking': 'True',
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
  'BusinessAcceptsCreditCards': 'True',
  'RestaurantsReservations': 'False',
  'WheelchairAccessible': 'True',
  'Caters': 'True',
  'OutdoorSeating': 'True',
  'RestaurantsGoodForGroups': 'True',
  'HappyHour': 'True',
  'BusinessAcceptsBitcoin': 'False',
  'RestaurantsPriceRange2': '2',
  'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}",
  'HasTV': 'True',
  'Alcohol': "'beer_and_wine'",
  'GoodFo

In [77]:
data[1]

{'business_id': 'tCbdrRPZA0oiIYSmHG3J0w',
 'name': 'Flying Elephants at PDX',
 'address': '7000 NE Airport Way',
 'city': 'Portland',
 'state': 'OR',
 'postal_code': '97218',
 'latitude': 45.5889058992,
 'longitude': -122.5933307507,
 'stars': 4.0,
 'review_count': 126,
 'is_open': 1,
 'attributes': {'RestaurantsTakeOut': 'True',
  'RestaurantsAttire': "u'casual'",
  'GoodForKids': 'True',
  'BikeParking': 'False',
  'OutdoorSeating': 'False',
  'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}",
  'Caters': 'True',
  'RestaurantsReservations': 'False',
  'RestaurantsDelivery': 'False',
  'HasTV': 'False',
  'RestaurantsGoodForGroups': 'False',
  'BusinessAcceptsCreditCards': 'True',
  'NoiseLevel': "u'average'",
  'ByAppointmentOnly': 'False',
  'RestaurantsPriceRange2': '2',
  'WiFi': "u'free'",
  'BusinessParking': "{'garage': True, 'street': False, 'validated':

In [59]:
d=[[json.dumps(ele) if type(ele)==dict else ele for ele in item.values() ] for item in data]

In [63]:
d

[['6iYb2HFDywm3zjuRg0shjw',
  'Oskar Blues Taproom',
  '921 Pearl St',
  'Boulder',
  'CO',
  '80302',
  40.0175444,
  -105.2833481,
  4.0,
  86,
  1,
  '{"RestaurantsTableService": "True", "WiFi": "u\'free\'", "BikeParking": "True", "BusinessParking": "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", "BusinessAcceptsCreditCards": "True", "RestaurantsReservations": "False", "WheelchairAccessible": "True", "Caters": "True", "OutdoorSeating": "True", "RestaurantsGoodForGroups": "True", "HappyHour": "True", "BusinessAcceptsBitcoin": "False", "RestaurantsPriceRange2": "2", "Ambience": "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': False, \'casual\': True}", "HasTV": "True", "Alcohol": "\'beer_and_wine\'", "GoodForMeal": "{\'dessert\': False, \'latenight\': False, \'lunch\': False, \'dinner\': False, \'brunch\': False, \'breakfast\': False

In [62]:
execute_bulk_insert("INSERT INTO raw_business VALUES %s",connect("yelp_db"),d)

INSERT INTO raw_business VALUES %s sucessfull
connection closed


In [25]:
data

[{'business_id': '6iYb2HFDywm3zjuRg0shjw',
  'name': 'Oskar Blues Taproom',
  'address': '921 Pearl St',
  'city': 'Boulder',
  'state': 'CO',
  'postal_code': '80302',
  'latitude': 40.0175444,
  'longitude': -105.2833481,
  'stars': 4.0,
  'review_count': 86,
  'is_open': 1,
  'attributes': {'RestaurantsTableService': 'True',
   'WiFi': "u'free'",
   'BikeParking': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
   'BusinessAcceptsCreditCards': 'True',
   'RestaurantsReservations': 'False',
   'WheelchairAccessible': 'True',
   'Caters': 'True',
   'OutdoorSeating': 'True',
   'RestaurantsGoodForGroups': 'True',
   'HappyHour': 'True',
   'BusinessAcceptsBitcoin': 'False',
   'RestaurantsPriceRange2': '2',
   'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}",
   'HasTV': 'True',
   'Alcohol': "

### FLATTEN_JSON

In [73]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [83]:
length=[]
for i in range(len(data)):
    data[i]=flatten_json(data[i])
    length.append(len(data[i]))

In [89]:
i=0
for item in length:
    if item>40:
        print(item, "  index: ",i)
        
    i+=1

41   index:  0
45   index:  29
47   index:  41
41   index:  42
42   index:  94
42   index:  113
42   index:  119
42   index:  147
43   index:  149
45   index:  165
41   index:  199
43   index:  248
41   index:  279
45   index:  295
42   index:  312
46   index:  323
43   index:  325
44   index:  341
45   index:  369
41   index:  385
45   index:  409
43   index:  410
41   index:  434
46   index:  474
44   index:  485
46   index:  488
43   index:  490
41   index:  516
43   index:  559
41   index:  587
42   index:  593
41   index:  618
46   index:  664
48   index:  669
41   index:  689
44   index:  703
41   index:  737
46   index:  738
41   index:  751
41   index:  769
47   index:  778
45   index:  825
41   index:  842
43   index:  869
47   index:  879
44   index:  899
43   index:  902
41   index:  922


In [94]:
data[500]

{'business_id': 'JyoZm72luJUJO0r2cwcPuw',
 'name': 'Southwest Airlines',
 'address': '1 Jeff Fuqua Blvd',
 'city': 'Orlando',
 'state': 'FL',
 'postal_code': '32827',
 'latitude': 28.4310857,
 'longitude': -81.3080758,
 'stars': 3.0,
 'review_count': 187,
 'is_open': 1,
 'attributes': None,
 'categories': 'Hotels & Travel, Airlines, Transportation',
 'hours_Monday': '0:0-0:0',
 'hours_Tuesday': '0:0-0:0',
 'hours_Wednesday': '0:0-0:0',
 'hours_Thursday': '0:0-0:0',
 'hours_Friday': '0:0-0:0',
 'hours_Saturday': '0:0-0:0',
 'hours_Sunday': '0:0-0:0'}

In [22]:
data[7]

{'business_id': 'jFYIsSb7r1QeESVUnXPHBw',
 'name': 'Boxwood Biscuit',
 'address': '740 S High St',
 'city': 'Columbus',
 'state': 'OH',
 'postal_code': '43206',
 'latitude': 39.947006523,
 'longitude': -82.997471,
 'stars': 4.5,
 'review_count': 11,
 'is_open': 1,
 'attributes': None,
 'categories': 'Breakfast & Brunch, Restaurants',
 'hours_Saturday': '8:0-14:0',
 'hours_Sunday': '8:0-14:0'}

In [11]:
from pandas.io.json import json_normalize